In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/sydney-house-prices/SydneyHousePrices.csv")

data.head()

In [ ]:
data.drop("Id",axis=1,inplace=True) 

df = pd.DataFrame(data)
print(df.shape)

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().values.any()

In [ ]:
df.isnull().sum()

In [ ]:
sns.pairplot(df)

In [ ]:
corrmat = df.corr() 
  
f, ax = plt.subplots(figsize =(12, 8)) 
sns.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 


In [ ]:
def boxplt(df, columns):
    for i in df.columns:
        if(df[i].dtype== np.int64 or df[i].dtype== np.float64):
            fig, ax =plt.subplots(figsize=(8,6))
            sns.boxplot(x = df[i])
boxplt(data,df.columns)    

In [ ]:
def find_outliers(df, columns):
    for i in df.columns:
        if(df[i].dtype== np.int64):
            q1=df[i].quantile(0.25)
            q3=df[i].quantile(0.75)
            iqr=q3-q1
            upper_bound= q3 + 1.5 * iqr
            lower_bound= q1 - 1.5 * iqr
            for k in range(len(df[i])):
                if df[i].iloc[k]<lower_bound:
                    df[i].iloc[k]=lower_bound
                if df[i].iloc[k]>upper_bound:
                    df[i].iloc[k]=upper_bound

In [ ]:
find_outliers(data,df.columns)

In [ ]:
#controling function of find_outlier 
def boxplt(df, columns):
    for i in df.columns:
        if(df[i].dtype== np.int64 or df[i].dtype== np.float64):
            fig, ax =plt.subplots(figsize=(8,6))
            sns.boxplot(x = df[i])
boxplt(data,df.columns)  

In [ ]:
#percent of missing values
def percent_missing(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    gf=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    gf= gf[gf["Percent"] > 0]
    f,ax =plt.subplots(figsize=(8,6))
    fig=sns.barplot(gf.index, gf["Percent"],color="purple",alpha=0.4)
    plt.xlabel('Features', fontsize=14)
    plt.ylabel('Percent of missing values', fontsize=16)
    plt.title('Missing values of feature', fontsize=16)
    return gf

In [ ]:
percent_missing(df)

In [ ]:
sns.heatmap(df.isnull(),yticklabels='auto',cmap= "viridis")

In [ ]:
index=[2,8]
for i in range(len(index)):
    print(df.groupby(df.columns[index[i]])["sellPrice"].count())
    print (f"***************\n")

In [ ]:
from sklearn.impute import SimpleImputer
columns = ['car','bed']
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[columns] = imp_mean.fit_transform(df[columns])
df.isnull().sum()

In [ ]:
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
df_fit = df.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
df_fit

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_fit.drop('sellPrice', axis=1), df_fit[ "sellPrice"], test_size=0.2)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


In [ ]:
sns.jointplot(x= "bed",y= "sellPrice",data=df_fit ,kind= "reg")

In [ ]:
k=df_fit[[ "bed"]]
l=df_fit[["sellPrice"]]
reg=LinearRegression()


In [ ]:
model=reg.fit(k,l)

In [ ]:
model